In [3]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np  
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
import cv2
from sklearn.utils import shuffle

from IPython.display import clear_output, display
import os.path
from os import path
import shutil
import PIL
from pathlib import Path
from matplotlib import pyplot as plt

from IPython.display import clear_output


import numpy as np

from PIL import ImageFont
from PIL import Image, ImageDraw
from PIL import ImageColor

from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

import random

In [5]:
import time
import shutil
import uuid
from shutil import copyfile

In [13]:
from collections import Counter

In [4]:
def read_label( path):
  f= open( path)
  d= f.read()
  f.close()
  return d 

def write_label( path, text):
  f= open( path, "w")
  f.write( text)
  f.close()

def show( img, name= 'noname'):
  plt.imshow( img)
  plt.title( name)
  plt.show()

def show_gimage( img, name= 'noname'):
  plt.imshow( img, cmap= 'gray')
  plt.title( name)
  plt.show()

def _load_image( path):
  # return cv2.imread( path, cv2.IMREAD_GRAYSCALE)
  return cv2.imread( path)
  
def write_image( path, img):
  cv2.imwrite( path, img)

# def resize( img, scale_percent= 60, verbose=False):
#   #scale_percent = 60 # percent of original size
#   width = int(img.shape[1] * scale_percent / 100)
#   height = int(img.shape[0] * scale_percent / 100)
#   dim = (width, height)
#   if verbose:
#     print( "scale", scale_percent)
#     print( "dim", dim)
#   # resize image
#   return cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 

In [6]:
# le os dados da base randomica

files= glob( r"C:\mestrado\datasets\carnaval\carnaval_8lines_shuffle_parts\labels\*.pgn")


In [7]:
len( files)

1426

In [8]:
labels= [read_label(f) for f in files]

In [17]:
labels

['e4',
 'c5',
 'Nf3',
 'd4',
 'Nf6',
 'Nc3',
 'd6',
 'Nbd7',
 'O-O',
 'Be7',
 'Be3',
 'O-O',
 'e4',
 'c5',
 'Nf3',
 'e6',
 'd4',
 'a6',
 'c4',
 'Qc7',
 'Nc3',
 'Nf6',
 'e4',
 'c5',
 'Nf3',
 'd6',
 'Bd7',
 'O-O',
 'Ngf6',
 'Re1',
 'e5',
 'c3',
 'Be7',
 'd3',
 'O-O',
 'e4',
 'Bc4',
 'O-O',
 'Nc3',
 'O-O',
 'a3',
 'd6',
 'd3',
 'Bg4',
 'Be3',
 'Nd4',
 'Nf3',
 'd5',
 'e6',
 'b3',
 'Be7',
 'Bb2',
 'O-O',
 'Bg2',
 'Nbd7',
 'O-O',
 'c6',
 'd3',
 'Re8',
 'e4',
 'c5',
 'Nf3',
 'Nc6',
 'd4',
 'g6',
 'Nc3',
 'Bg7',
 'Be3',
 'Nf6',
 'Bc4',
 'O-O',
 'Bb3',
 'd6',
 'd4',
 'c6',
 'Nf3',
 'd5',
 'c3',
 'Nf6',
 'Bf4',
 'g6',
 'Bg7',
 'Nbd2',
 'O-O',
 'Bd3',
 'Nbd7',
 'Ne5',
 'Nh5',
 'd4',
 'd5',
 'Bf4',
 'Nf6',
 'e3',
 'c5',
 'c3',
 'Qb6',
 'Qb3',
 'c4',
 'Qc2',
 'g6',
 'Nd2',
 'Nc6',
 'e4',
 'e6',
 'e4',
 'e5',
 'Nf3',
 'd6',
 'Nc3',
 'Nc6',
 'Bc4',
 'Be7',
 'h3',
 'Nf6',
 'g3',
 'd5',
 'Bg2',
 'Nf6',
 'c4',
 'c6',
 'Nf3',
 'a4',
 'Bb7',
 'b3',
 'e4',
 'e5',
 'Nf3',
 'Nc6',
 'Bc4',
 'Nf6',
 'Ng5',
 'd

In [10]:
flat_tokens = [t for tlist in labels for t in tlist]

In [11]:
len( flat_tokens)

3675

In [18]:
count= Counter( labels)

In [19]:
count

Counter({'e4': 64,
         'c5': 58,
         'Nf3': 86,
         'd4': 83,
         'Nf6': 85,
         'Nc3': 63,
         'd6': 36,
         'Nbd7': 13,
         'O-O': 103,
         'Be7': 33,
         'Be3': 16,
         'e6': 52,
         'a6': 27,
         'c4': 37,
         'Qc7': 9,
         'Bd7': 12,
         'Ngf6': 2,
         'Re1': 6,
         'e5': 37,
         'c3': 20,
         'd3': 18,
         'Bc4': 12,
         'a3': 10,
         'Bg4': 8,
         'Nd4': 1,
         'd5': 63,
         'b3': 11,
         'Bb2': 8,
         'Bg2': 17,
         'c6': 19,
         'Re8': 2,
         'Nc6': 59,
         'g6': 24,
         'Bg7': 21,
         'Bb3': 5,
         'Bf4': 14,
         'Nbd2': 11,
         'Bd3': 17,
         'Ne5': 2,
         'Nh5': 1,
         'e3': 23,
         'Qb6': 8,
         'Qb3': 3,
         'Qc2': 7,
         'Nd2': 6,
         'h3': 10,
         'g3': 14,
         'a4': 4,
         'Bb7': 7,
         'Ng5': 1,
         'Na5': 2,
         'Be2

In [25]:
len( [key for (key, value) in count.items()])

102